In [63]:
import struct
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import math
data = open("statistics_03500_repo.bin", "rb").read()

In [64]:
#Leitura de parâmetros gerais
(my,mz,ubulk,utau,fnu,tstat)=struct.unpack("iidddd", data[:40])

In [65]:
#Leitura de coordenadas
n=my*8+40
y=struct.unpack("d"*my, data[40:n])
m=n
n+=mz*8
z=struct.unpack("d"*mz, data[m:n])

In [66]:
#Leitura das velocidades médias
m=n
n+=my*mz*8
um=struct.unpack("d"*my*mz, data[m:n])
um=np.reshape(um,(my,mz))
m=n
n+=my*mz*8
vm=struct.unpack("d"*my*mz, data[m:n])
vm=np.reshape(vm,(my,mz))
m=n
n+=my*mz*8
wm=struct.unpack("d"*my*mz, data[m:n])
wm=np.reshape(wm,(my,mz))

In [67]:
#Leitura dos termos do tensor de Reynolds
m=n
n+=my*mz*8
uu=struct.unpack("d"*my*mz, data[m:n])
uu=np.reshape(uu,(my,mz))
m=n
n+=my*mz*8
uv=struct.unpack("d"*my*mz, data[m:n])
uv=np.reshape(uv,(my,mz))
m=n
n+=my*mz*8
uw=struct.unpack("d"*my*mz, data[m:n])
uw=np.reshape(uw,(my,mz))
m=n
n+=my*mz*8
vv=struct.unpack("d"*my*mz, data[m:n])
vv=np.reshape(vv,(my,mz))
m=n
n+=my*mz*8
vw=struct.unpack("d"*my*mz, data[m:n])
vw=np.reshape(vw,(my,mz))
m=n
n+=my*mz*8
ww=struct.unpack("d"*my*mz, data[m:n])
ww=np.reshape(ww,(my,mz))

In [68]:
ReStress=np.array([[uu,uv,uw],[uv,vv,vw],[uw,vw,ww]])
VelMedia=np.array([um,vm,wm])
ReStress=np.transpose(ReStress,(2,3,0,1))
VelMedia=np.transpose(VelMedia,(1,2,0))

In [89]:
def AnisotropyTensor(x):
    k=(np.trace(x,axis1=2,axis2=3)/2)
    a=np.divide(x,2*k[:,:,np.newaxis,np.newaxis])-((1/3)*np.eye(3))
    alambda,aEpsilon = LA.eig(a)
    II=alambda[:,:,0]*alambda[:,:,0]+alambda[:,:,1]*alambda[:,:,1]+alambda[:,:,0]*alambda[:,:,1]
    III=-alambda[:,:,0]*alambda[:,:,1]*(alambda[:,:,0]+alambda[:,:,1])
    epsilon=np.power(np.abs(III/2),1/3)* np.sign(III/2)
    eta=np.sqrt(II/3)
    xb=alambda[:,:,0]-alambda[:,:,1]+(3/2)*alambda[:,:,2]+(1/2)
    yb=(np.sqrt(3)/2)*(3*alambda[:,:,2]+1)
    return a,k,alambda,aEpsilon,III,II,epsilon,eta,xb,yb
def EulerAngles(x,string):
    #euler angles "x-convection" based on cartesian local system
    SMALL=1e-16
    #Xl = Z1 x Z
    #normalizing
    x=np.einsum('ijkl,ijl->ijkl',x,1/LA.norm(x,axis=2))
    #x=np.array([[x[0,0]/LA.norm(x[:,0]),x[0,1]/LA.norm(x[:,1]),x[0,2]/LA.norm(x[:,2])],
    #            [x[1,0]/LA.norm(x[:,0]),x[1,1]/LA.norm(x[:,1]),x[1,2]/LA.norm(x[:,2])],
    #            [x[2,0]/LA.norm(x[:,0]),x[2,1]/LA.norm(x[:,1]),x[2,2]/LA.norm(x[:,2])]])
    Xl=np.array([x[:,:,1,2],-x[:,:,0,2],0*x[:,:,2,2]])
    Xl=np.transpose(Xl,(1,2,0))
    #X1’x = X’·X1 = X’x ·X1x + X’y ·X1y + X’z ·X1z = Z1y ·X1x – Z1x ·X1y
    X1lx=np.einsum('ijk,ijk->ij', Xl, x[:,:,:,0])
    X1ly=np.einsum('ijk,ijk->ij', Xl, x[:,:,:,1])
    Z1xy=np.array([x[:,:,0,2],x[:,:,1,2],0*x[:,:,1,2]])
    Z1xy=np.transpose(Z1xy,(1,2,0))
    Z1xy=LA.norm(Z1xy,axis=2)
    angles=np.where(Z1xy>SMALL,
    np.array([np.arctan2(X1ly, X1lx),np.arctan2(Z1xy,x[:,:,2,2]),-1.*np.arctan2(-x[:,:,0,2],x[:,:,1,2])]),
    np.array([0*np.arctan2(X1ly, X1lx),np.where(np.abs(x[:,:,2,2])>SMALL),-1.*np.arctan2(X1ly, X1lx)]))
    if Z1xy>SMALL:
        alpha=np.arctan2(X1ly, X1lx)
        beta=np.arctan2(Z1xy,x[:,:,2,2])
        gamma=-1.*np.arctan2(-x[:,:,0,2],x[:,:,1,2])
    else:
        alpha=0.
        if np.abs(x[:,:,2,2])>SMALL:
            beta=0.
        else:
            beta=np.pi
        gamma=-1.*np.arctan2(X1ly, X1lx)
    if string=='graus':
        return np.array([alpha,beta,gamma])*360/(2*np.pi)
    else:
        return np.array([alpha,beta,gamma])
def EulerInverse(alfa,beta,gamma,base):
    A=np.array([[np.cos(alfa),np.sin(alfa),0],[-np.sin(alfa),np.cos(alfa),0],[0,0,1]])
    B=np.array([[1,0,0],[0,np.cos(beta),np.sin(beta)],[0,-np.sin(beta),np.cos(beta)]])
    C=np.array([[np.cos(gamma),np.sin(gamma),0],[-np.sin(gamma),np.cos(gamma),0],[0,0,1]])
    CB=np.einsum('ij,jk->ik', C, B)
    CBA=np.einsum('ij,jk->ik', CB, A)
    QBase=np.einsum('ij,jk->ik', CBA, base)
    return QBase

In [90]:
EulerAngles(AnisotropyTensor(ReStress[:2,:2,:,:])[3],'rad')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()